In [2]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random
import networkx as nx


from icecream import ic

logging.basicConfig(level=logging.DEBUG)

## Read the file and properly initialize data structures

In [3]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [4]:
#Used as stopping criteria for the EA algorithm
best_results = [1345.54,4172.76,32722,39016]
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    cluster_set = set(tsp[:-1])  # Escludi l'ultima città (uguale alla prima)
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost if cluster_set.issubset(set(range(len(CITIES)))) else float('inf')

def fitness(solution):
    return -tsp_cost(solution)

median = np.median(DIST_MATRIX.reshape(1, -1))
ic(median)
DIST_MATRIX[DIST_MATRIX > median] = np.inf
G = nx.Graph()
for c1, c2 in combinations(CITIES.itertuples(), 2):
    if DIST_MATRIX[c1.Index, c2.Index] <= median:
        G.add_edge(c1, c2)
nx.is_connected(G)


ic| median: np.float64(367.0694334013403)


True

## Path from A to B
Randomly extract two cities in the Italian csv file of cities and find a path between them, consider two cities A and B connected only if their distance is not np.inf().

In [ ]:
# Assuming CITIES is a list of cities and DIST_MATRIX is a distance matrix
A, B = random.sample(range(len(CITIES)), 2)
print("Starting city:", A, "Destination city:", B)


#The graph is connected, there exists for sure a path from A to B
#Try to find it with Dijkstra
def dijkstra_path(A, B, DIST_MATRIX):
    n = len(DIST_MATRIX)
    dist = [np.inf] * n
    dist[A] = 0
    prev = [None] * n
    unvisited = set(range(n))

    while unvisited:
        current = min(unvisited, key=lambda x: dist[x])
        unvisited.remove(current)

        if dist[current] == np.inf or current == B:
            break

        for neighbor in range(n):
            if neighbor in unvisited and DIST_MATRIX[current, neighbor] != np.inf:
                alt = dist[current] + DIST_MATRIX[current, neighbor]
                if alt < dist[neighbor]:
                    dist[neighbor] = alt
                    prev[neighbor] = current

    path = []
    u = B
    if prev[u] is not None or u == A:
        while u is not None:
            path.insert(0, u)
            u = prev[u]
    return path, dist[B]

#The solution found with Dijkstra should be the best one in terms of costs
path,cost = dijkstra_path(A,B,DIST_MATRIX)
print(path,cost)


Starting city: 16 Destination city: 36
[16, 36] 344.66168759909914
